In [10]:
import incense
from common.helper.run import checkpointsById
from common.helper.config import parseMongoConfig
from common.model import CNNIMU
from ptflops import get_model_complexity_info
import torch
import logging
import io
loader = incense.ExperimentLoader(**parseMongoConfig('../config.ini', adapt='IncenseExperimentLoader'))

# Load best Experiments ($wF_1$)

In [2]:
queries = {
    'opportunity': {
        '$and': [
            { 'experiment.name': 'CNN-IMU_Opportunity-Locomotion' },
            { 'status': 'COMPLETED' },
            { 'info.metrics': {'$elemMatch': {'name' : 'test/wf1'}} }
        ]
    },
    'lara': {
        '$and': [
            { 'experiment.name': 'CNN-IMU_LARa' },
            { 'status': 'COMPLETED' },
            { '_id': {'$gte' : 215} },
            { 'info.metrics': {'$elemMatch': {'name' : 'test/wf1'}} }
        ]
    },
    'pamap2': {
        '$and': [
            { 'experiment.name': 'CNN-IMU_Pamap2(activity_labels)' },
            { 'status': 'COMPLETED' },
            { '_id': {'$gte' : 183} },
            { 'info.metrics': {'$elemMatch': {'name' : 'test/wf1'}} }
        ]
    }
}
experiments = { k: max(loader.find(q), key=lambda e: e.metrics['test/wf1'].max()) for k, q in queries.items()}

# Estimate Multiply Accumulate Operations with `ptflops`

In [18]:
for n, e in experiments.items():
  ckpt = checkpointsById(root='../logs/checkpoints', run_id=e.to_dict()['_id'])['best_wf1']

  logging.disable(level=logging.INFO)
  model = CNNIMU.load_from_checkpoint(ckpt)
  logging.disable(level=logging.NOTSET)
  model.eval()

  def batch_ctor(*_):
    return {
        'imu_x': [
            torch.rand(size=(1, model.hparams['sample_length'], imu_size))
            for imu_size in model.hparams['imu_sizes']
        ]
    }
  
  macs, params = get_model_complexity_info(model, input_res=(0,), input_constructor=batch_ctor, ost=io.StringIO())

  print(f'CNNIMU {n:<15} has {params:<6} parameters and needs {macs:<10} for inference')



Warning! No positional inputs found for a module, assuming batch size is 1.
CNNIMU opportunity     has 15.5 M parameters and needs 147.63 MMac for inference
Warning! No positional inputs found for a module, assuming batch size is 1.
CNNIMU lara            has 26.2 M parameters and needs 151.18 MMac for inference
Warning! No positional inputs found for a module, assuming batch size is 1.
CNNIMU pamap2          has 34.07 M parameters and needs 200.87 MMac for inference
